<a href="https://colab.research.google.com/github/Mohammed0Arfath/Qiskit_Deustch_Jozsa/blob/main/Qiskit_Deustch_Jozsa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install qiskit
!pip install qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 62.4 MB/s eta 0:00:00


In [2]:
# Deutsch–Jozsa Algorithm using Qiskit 2.x
# Compatible with Qiskit >= 2.0.0

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt


In [3]:
# ---------- ORACLES ----------
def oracle_constant(qc, ancilla, value=0):
    """Constant oracle: f(x)=0 or f(x)=1"""
    if value == 1:
        qc.x(ancilla)


def oracle_balanced_parity(qc, inputs, ancilla):
    """Balanced oracle: f(x) = x0 XOR x1 XOR ... XOR xn"""
    for q in inputs:
        qc.cx(q, ancilla)




In [4]:
# ---------- DEUTSCH–JOZSA CIRCUIT ----------
def deutsch_jozsa_circuit(n, oracle_func, *oracle_args):
    """
    n: number of input qubits
    oracle_func: oracle function to modify the circuit
    oracle_args: extra arguments for oracle
    """
    qreg = QuantumRegister(n + 1, "q")
    creg = ClassicalRegister(n, "c")
    qc = QuantumCircuit(qreg, creg)

    inputs = list(range(n))
    ancilla = n

    # Step 1: Initialize |0...0>|1>
    qc.x(ancilla)

    # Step 2: Apply Hadamard to all qubits
    qc.h(qreg)

    # Step 3: Oracle
    oracle_func(qc, *oracle_args)

    # Step 4: Apply Hadamard to input qubits
    for q in inputs:
        qc.h(q)

    # Step 5: Measure only input qubits
    qc.measure(inputs, creg)

    return qc


In [5]:
# ---------- EXECUTION ----------
def run_dj(qc):
    """Run Deutsch–Jozsa circuit on AerSimulator"""
    simulator = AerSimulator()
    tqc = transpile(qc, simulator)
    job = simulator.run(tqc, shots=1024)
    result = job.result()
    counts = result.get_counts()

    print("Measurement counts:", counts)
    plot_histogram(counts)
    plt.show()

    n = qc.num_clbits
    if counts.get("0" * n, 0) == 1024:
        print("✅ Function is CONSTANT")
    else:
        print("✅ Function is BALANCED")


In [6]:
# ---------- MAIN ----------
if __name__ == "__main__":
    n = 3  # number of input qubits

    print("\n=== Constant Oracle (f(x)=0) ===")
    qc_const = deutsch_jozsa_circuit(
        n, oracle_constant, n, 0
    )
    print(qc_const.draw(fold=-1))
    run_dj(qc_const)

    print("\n=== Balanced Oracle (Parity) ===")
    qc_balanced = deutsch_jozsa_circuit(
        n, oracle_balanced_parity, list(range(n)), n
    )
    print(qc_balanced.draw(fold=-1))
    run_dj(qc_balanced)



=== Constant Oracle (f(x)=0) ===
     ┌───┐┌───┐┌─┐      
q_0: ┤ H ├┤ H ├┤M├──────
     ├───┤├───┤└╥┘┌─┐   
q_1: ┤ H ├┤ H ├─╫─┤M├───
     ├───┤├───┤ ║ └╥┘┌─┐
q_2: ┤ H ├┤ H ├─╫──╫─┤M├
     ├───┤├───┤ ║  ║ └╥┘
q_3: ┤ X ├┤ H ├─╫──╫──╫─
     └───┘└───┘ ║  ║  ║ 
c: 3/═══════════╩══╩══╩═
                0  1  2 
Measurement counts: {'000': 1024}
✅ Function is CONSTANT

=== Balanced Oracle (Parity) ===
     ┌───┐          ┌───┐     ┌─┐           
q_0: ┤ H ├───────■──┤ H ├─────┤M├───────────
     ├───┤       │  └───┘┌───┐└╥┘     ┌─┐   
q_1: ┤ H ├───────┼────■──┤ H ├─╫──────┤M├───
     ├───┤       │    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_2: ┤ H ├───────┼────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_3: ┤ X ├┤ H ├┤ X ├┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘└───┘└───┘ ║       ║  ║ 
c: 3/══════════════════════════╩═══════╩══╩═
                               0       1  2 
Measurement counts: {'111': 1024}
✅ Function is BALANCED


In [11]:
# ==========================
# INSTALL DEPENDENCIES
# ==========================
!pip install qiskit qiskit_aer qiskit_ibm_runtime --quiet

# ==========================
# IMPORTS
# ==========================
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
from qiskit.quantum_info import Operator
import matplotlib.pyplot as plt

# ==========================
# ORACLES
# ==========================
def oracle_constant(qc, ancilla, value=0):
    """Constant oracle: f(x)=0 or f(x)=1"""
    if value == 1:
        qc.x(ancilla)

def oracle_balanced_parity(qc, inputs, ancilla):
    """Balanced oracle: f(x)=x0 XOR x1 XOR ... XOR xn"""
    for q in inputs:
        qc.cx(q, ancilla)

def oracle_balanced_custom(qc, inputs, ancilla):
    """
    Custom balanced oracle:
    Flip ancilla if first two qubits are both 1 (half of all 2^n inputs)
    f(x) = x0 AND x1
    """
    if len(inputs) < 2:
        raise ValueError("Need at least 2 input qubits for this oracle")
    qc.ccx(inputs[0], inputs[1], ancilla)

# ==========================
# DEUTSCH–JOZSA CIRCUIT
# ==========================
def deutsch_jozsa_circuit(n, oracle_func, *oracle_args):
    """
    Build Deutsch–Jozsa circuit for n input qubits.
    """
    qreg = QuantumRegister(n + 1, "q")
    creg = ClassicalRegister(n, "c")
    qc = QuantumCircuit(qreg, creg)

    inputs = list(range(n))
    ancilla = n

    # Step 1: Initialize |0...0>|1>
    qc.x(ancilla)

    # Step 2: Apply Hadamard to all qubits
    qc.h(qreg)

    # Step 3: Oracle
    oracle_func(qc, *oracle_args)

    # Step 4: Apply Hadamard to input qubits
    for q in inputs:
        qc.h(q)

    # Step 5: Measure only input qubits
    qc.measure(inputs, creg)

    return qc

# ==========================
# EXECUTION HELPERS
# ==========================
def run_dj(qc):
    """Run Deutsch–Jozsa circuit on AerSimulator"""
    simulator = AerSimulator()
    tqc = transpile(qc, simulator)
    job = simulator.run(tqc, shots=1024)
    result = job.result()
    counts = result.get_counts()
    plot_histogram(counts)
    plt.show()
    print("Counts:", counts)

    n = qc.num_clbits
    if counts.get("0" * n, 0) == 1024:
        print("✅ Function is CONSTANT\n")
    else:
        print("✅ Function is BALANCED\n")

# ==========================
# CUSTOM NOISE SIMULATION
# ==========================
def create_simple_noise_model():
    noise_model = NoiseModel()

    # Add simple depolarizing noise to all single & two-qubit gates
    single_qubit_error = depolarizing_error(0.02, 1)
    two_qubit_error = depolarizing_error(0.05, 2)

    noise_model.add_all_qubit_quantum_error(single_qubit_error, ['h', 'x', 'u1', 'u2', 'u3'])
    noise_model.add_all_qubit_quantum_error(two_qubit_error, ['cx'])
    return noise_model

def run_dj_noisy(qc):
    """Run Deutsch–Jozsa circuit with a simple noise model"""
    noise_model = create_simple_noise_model()
    simulator = AerSimulator(noise_model=noise_model)
    tqc = transpile(qc, simulator)
    job = simulator.run(tqc, shots=1024)
    result = job.result()
    counts = result.get_counts()

    plot_histogram(counts)
    plt.show()
    print("Noisy Counts:", counts)
    print("👀 Observe: noise distorts the ideal results.\n")

# ==========================
# MAIN EXECUTION
# ==========================
if __name__ == "__main__":

    # ---------- TASK 1: CUSTOM BALANCED ORACLE ----------
    print("\n🔹 TASK 1 — Custom Balanced Oracle (f(x)=x0 AND x1)")
    n = 3
    qc_custom = deutsch_jozsa_circuit(n, oracle_balanced_custom, list(range(n)), n)
    print(qc_custom.draw(fold=-1))
    run_dj(qc_custom)

    # ---------- TASK 2: VARY INPUT SIZE ----------
    print("\n🔹 TASK 2 — Vary Input Qubits (n=2, 4, 5)")
    for n in [2, 4, 5]:
        print(f"\n=== n = {n} ===")
        qc = deutsch_jozsa_circuit(n, oracle_balanced_custom, list(range(n)), n)
        print(qc.draw(fold=-1))
        run_dj(qc)
        print(f"Circuit Depth: {qc.depth()}, Width: {qc.width()}")

    # ---------- TASK 3: ADD NOISE ----------
    print("\n🔹 TASK 3 — Noise Simulation (Custom Noise Model)")
    qc_noisy = deutsch_jozsa_circuit(3, oracle_balanced_custom, list(range(3)), 3)
    run_dj_noisy(qc_noisy)

    # ---------- TASK 4: RUN ON IBM BACKEND ----------
    print("\n🔹 TASK 4 — Run on IBM Quantum Device / Simulator")
    try:
        service = QiskitRuntimeService(channel="ibm_quantum")
        backend = service.backend("ibmq_qasm_simulator")  # change to a real device if you have access
        sampler = SamplerV2(backend)

        tqc = transpile(qc_custom, backend)
        job = sampler.run([tqc])
        result = job.result()
        counts = result[0].data.meas.get_counts()
        plot_histogram(counts)
        plt.show()
        print("IBM Backend Counts:", counts)
    except Exception as e:
        print("⚠️ IBM Quantum execution skipped:", e)
        print("Tip: Run 'QiskitRuntimeService.save_account()' once to link your IBM account.")

    # ---------- TASK 5: ORACLE UNITARY ANALYSIS ----------
    print("\n🔹 TASK 5 — Oracle Circuit & Unitary Matrix")
    oracle_qc = QuantumCircuit(4)  # 3 inputs + 1 ancilla
    oracle_balanced_custom(oracle_qc, [0, 1, 2], 3)
    gate = oracle_qc.to_gate(label="U_f")

    print("Oracle Gate Definition:\n", gate.definition)
    print("\nOracle Matrix Representation:")
    print(Operator(oracle_qc).data)
    print("\nInterpretation:")
    print("This oracle flips the ancilla only when x0=x1=1 → f(x)=x0·x1.")
    print("Hence, half of the 2^n inputs cause ancilla flip → Balanced.\n")



🔹 TASK 1 — Custom Balanced Oracle (f(x)=x0 AND x1)
     ┌───┐          ┌───┐┌─┐   
q_0: ┤ H ├───────■──┤ H ├┤M├───
     ├───┤       │  ├───┤└╥┘┌─┐
q_1: ┤ H ├───────■──┤ H ├─╫─┤M├
     ├───┤┌───┐  │  └┬─┬┘ ║ └╥┘
q_2: ┤ H ├┤ H ├──┼───┤M├──╫──╫─
     ├───┤├───┤┌─┴─┐ └╥┘  ║  ║ 
q_3: ┤ X ├┤ H ├┤ X ├──╫───╫──╫─
     └───┘└───┘└───┘  ║   ║  ║ 
c: 3/═════════════════╩═══╩══╩═
                      2   0  1 
Counts: {'011': 235, '010': 257, '000': 257, '001': 275}
✅ Function is BALANCED


🔹 TASK 2 — Vary Input Qubits (n=2, 4, 5)

=== n = 2 ===
     ┌───┐          ┌───┐┌─┐   
q_0: ┤ H ├───────■──┤ H ├┤M├───
     ├───┤       │  ├───┤└╥┘┌─┐
q_1: ┤ H ├───────■──┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐└───┘ ║ └╥┘
q_2: ┤ X ├┤ H ├┤ X ├──────╫──╫─
     └───┘└───┘└───┘      ║  ║ 
c: 2/═════════════════════╩══╩═
                          0  1 
Counts: {'01': 253, '11': 246, '00': 260, '10': 265}
✅ Function is BALANCED

Circuit Depth: 5, Width: 5

=== n = 4 ===
     ┌───┐          ┌───┐   ┌─┐   
q_0: ┤ H ├──────